Le code suivant vise à retrouver les résultats du mémoire de maîtrise de Ferdinand Willemin (2025).

In [20]:
# Modules à importer
using Random

include("src/dataGen.jl");
include("src/sqavi.jl");
include("src/mcmc.jl");
include("src/plotting.jl");

# CHAPITRE 6. Résultats sur données de simulation

## Charger des anciens résultats
Exécuter la cellule ci-dessous pour charger les sorties SQAVI (dans la variable `res`) et MCMC (dans la variable `chain`) d'un calcul précédent.  

Seul le nom du dossier doit être spécifié, étant donné que les noms de fichiers sont prédéfinis (respectivement `sqavires.dat` et `mcmc_chain.dat`).

In [73]:
# Exécution sur les données simulées pour une grille de taille 200x200.
loadFolder = "results/200x200";

res = loadRes(loadFolder);
chain = loadChain(loadFolder);

**IMPORTANT** : Si les résultats sont chargés, alors **on peut sauter les sections 6.1, 6.2 et 6.3**.

**IMPORTANT** : Si on veut tracer de nouveaux graphiques à partir des résultats chargés, alors il faut tout de même spécifier un `saveFolder` (cf cellules suivantes).

## Créer de nouveaux résultats

Il faut spécifier le dossier dans lequel on veut sauvegarder les sorties SQAVI et MCMC.

**ATTENTION** : il faut que le dossier ait été créé avec la structure suivante afin de pouvoir enregistrer les tracés :
```
.
├── exemple
    └── plots
```

In [22]:
saveFolder = "results/exemple";

## 6.1. Les données

Génération des données de simulation.

Taille de la grille à générer.

In [13]:
m₂ = 5;
m₁ = 5;
m = m₁ * m₂;

Génère la grille des vraies valeurs de $\bm\mu$, $\bm\phi$ et $\xi$.

In [19]:
Random.seed!(300); # Graine de génération des nombres aléatoires.
Fmu = iGMRF(m₁, m₂, 1, 1);
Fphi = iGMRF(m₁, m₂, 1, 10);
gridTarget = generateTargetGrid(Fmu, Fphi);
gridTarget[:, :, 1] = gridTarget[:, :, 1] .+ 40.0; # Les μ sont centrés en 40.
gridTarget[:, :, 2] = gridTarget[:, :, 2] .+ 1.0; # Les ϕ sont centrés en 1.
gridTarget[:, :, 3] = gridTarget[:, :, 3] .+ 0.05; # On fixe ξ = .05
nobs = 100; # Nombre d'observations extrêmes par cellule.
data = generateData(gridTarget, nobs); # Données sous forme de Vector{Vector{Float64}}.

## 6.2. Estimations SQAVI

Modifier la variable `nEpochMax` pour spécifier le nombre maximal d'époques à effectuer avant de s'arrêter (cas où la convergence n'est pas atteinte).

Modifier la variable `epochSize` pour spécifier le nombre d'itérations par époques.

On ne fournit pas de valeurs initiales car celles-ci sont fixées de la sorte :
- $\mu$ et $\phi$ sont initialisés à leur estimations de maximum de vraisemblance.
- $\xi$ est initialisé à la moyenne des estimations de maximum de vraisemblance sur toute la grille.
- $b_u$ et $b_v$ sont initialisés de sorte que $\kappa^{(0)}_u = 1$ et $\kappa^{(0)}_v = 1$. 

In [46]:
nEpochMax = 10;
epochSize = 1;

spatialScheme = Dict(
    :m => m,
    :Fmu => Fmu,
    :Fphi => Fphi,
    :data => data,
);

Exécution de SQAVI.

In [47]:
ϵ = 0.001 # Critère d'arrêt.

res = runSQAVI(nEpochMax, epochSize, spatialScheme, ϵ, saveFolder=saveFolder);

Itération 0...
Itération 1...
Itération 2...
Itération 3...
Itération 4...
Itération 5...
Itération 6...
Itération 7...
Itération 8...
Itération 9...
L'algorithme a convergé !


Tracés

In [51]:
controlCell = 1; # Numéro de la cellule témoin.

plotConvergenceCriterion(res.MCKL, saveFolder=saveFolder);
plotTraceSQAVI(res.traces[:muMean][controlCell, :], "μ$controlCell", saveFolder=saveFolder);
plotTraceSQAVI(res.traces[:phiMean][controlCell, :], "ϕ$controlCell", saveFolder=saveFolder);
plotTraceSQAVI(res.traces[:xiMean], "ξ", saveFolder=saveFolder);
trace = res.traces[:kappaUparams][1, :] ./ res.traces[:kappaUparams][2, :];
plotTraceSQAVI(trace, "κᵤ", saveFolder=saveFolder);
trace = res.traces[:kappaVparams][1, :] ./ res.traces[:kappaVparams][2, :];
plotTraceSQAVI(trace, "κᵥ", saveFolder=saveFolder);

## 6.3. Estimations MCMC

### Configuration MCMC
- Ne pas toucher à la variable `datastructure`.
- `niter` contrôle le nombre d'itérations MCMC.
- `stepsize` contrôle la variance instrumentale de chaque paramètre -> à adapter en fonction du taux d'acceptation. 


In [63]:
datastructure = Dict(
    :Y => data,
    :Fmu => Fmu,
    :Fphi => Fphi,
);

niter = 10000;

stepsize = Dict(
    :μ => 1.25,
    :ϕ => .35,
    :ξ => .05,
);

### Exécution

In [64]:
chain = mcmc(datastructure, niter, stepsize, saveFolder=saveFolder);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:08


Taux d'acceptation.

In [65]:
changerate(chain)

             Change Rate
          μ1       0.356
          μ2       0.256
          μ3       0.254
          μ4       0.204
          μ5       0.198
          μ6       0.313
          μ7       0.242
          μ8       0.226
          μ9       0.212
         μ10       0.227
         μ11       0.281
         μ12       0.227
         μ13       0.238
         μ14       0.235
         μ15       0.249
         μ16       0.238
         μ17       0.216
         μ18       0.219
         μ19       0.224
         μ20       0.257
         μ21       0.202
         μ22       0.247
         μ23       0.204
         μ24       0.205
         μ25       0.245
          ϕ1       0.228
          ϕ2       0.234
          ϕ3       0.224
          ϕ4       0.235
          ϕ5       0.235
          ϕ6       0.230
          ϕ7       0.221
          ϕ8       0.217
          ϕ9       0.224
         ϕ10       0.236
         ϕ11       0.222
         ϕ12       0.216
         ϕ13       0.214
         ϕ14       0.221


### Tracés

Modifier la valeur de `varName` pour afficher la trace d'un autre paramètre.

Le tracé apparaîtra dans le dossier de sauvegarde.

In [67]:
varName = "ξ";

plotTraceMCMC(chain, varName, saveFolder=saveFolder);

## 6.4. Comparaison SQAVI et MCMC

### Tracés de comparaison

On a besoin de spécifier l'itération de chauffe du MCMC (`warmingSize`) pour afficher un échantillon *a posteriori* satisfaisant.

`controlCell` indique la cellule témoin pour laquelle on va regarder $\mu$ et $\phi$, il faut qu'elle soit comprise entre $1$ et $m$.

**ATTENTION** : les domaines de définition des lois d'approximations doivent être spécifiés dans le fichier `src/plotting.jl` pour chaque tracé, sans quoi les densités d'approximation n'apparaîtront pas convenablement.

In [77]:
controlCell = 2;
warmingSize = 1000;

plotSQAVIvsMCMC(controlCell, sqaviRes=res, mcmcChain=chain, warmingSize=warmingSize, saveFolder=saveFolder);

### Estimations MCMC

In [ ]:
xi_mcmc = mean(chain[:, "ξ", :].value[warmingSize:end]);
kappa_v_mcmc = mean(chain[:, "κᵥ", :].value[warmingSize:end]);
kappa_u_mcmc = mean(chain[:, "κᵤ", :].value[warmingSize:end]);
mu_mcmc = mean(chain[:, "μ$controlCell", :].value[warmingSize:end]);
phi_mcmc = mean(chain[:, "ϕ$controlCell", :].value[warmingSize:end]);

### Estimations SQAVI

In [75]:
xi_sqavi = res.traces[:xiMean][end];
kappa_v_sqavi = res.traces[:kappaVparams][1, end] ./ res.traces[:kappaVparams][2, end];
kappa_u_sqavi = res.traces[:kappaUparams][1, end] ./ res.traces[:kappaUparams][2, end];
mu_sqavi = res.traces[:muMean][controlCell, end];
phi_sqavi = res.traces[:phiMean][controlCell, end];

### Distance absolue entre les estimations

In [79]:
abs(kappa_v_mcmc - kappa_v_sqavi)

0.22750998320181104

# CHAPITRE 7. Résultats sur données de réanalyse

In [2]:
include("casr/src/utils.jl");
include("casr/src/preprocessing.jl");
include("casr/src/montreal.jl");

### Données

In [3]:
DATA_FOLDER = "casr/data";
RAW_FOLDER = "casr/raw";

### Pré-traitement

In [4]:
datasetMaxima = load_array3d("$DATA_FOLDER/preprocessed/maxima.bin");

# Réduction de la taille pour délimiter plus finement le Québec
(m₁, m₂) = (210, 175);
m = prod((m₁, m₂));
quebecMaxima = datasetMaxima[1:m₁, 1:m₂, :]

data = preprocessMaxima(quebecMaxima);

Cellule témoin (Montréal)

In [5]:
lats = load_matrix("casr/data/viz/lats.bin")[1:m₁, 1:m₂];
lons = load_matrix("casr/data/viz/lons.bin")[1:m₁, 1:m₂] .- 360;

MTLcell = findMontreal(lats, lons);
MTLcell

Coordonnées du point de plus proche de Montréal : [45.49225, -73.61365]


33515

Pour la visualisation des données et de la cellule témoin, voir `visualisation.ipynb`.

## Charger des anciens résultats
Exécuter la cellule ci-dessous pour charger les sorties SQAVI (dans la variable `res`) et MCMC (dans la variable `chain`) d'un calcul précédent.  

Seul le nom du dossier doit être spécifié, étant donné que les noms de fichiers sont prédéfinis (respectivement `sqavires.dat` et `mcmc_chain.dat`).

In [6]:
# Exécution sur les données simulées pour une grille de taille 200x200.
loadFolder = "results/casr";

res = loadRes(loadFolder);
chain = loadChain(loadFolder);

**IMPORTANT** : Si on veut tracer de nouveaux graphiques à partir des résultats chargés, alors il faut tout de même spécifier un `saveFolder` (cf cellules suivantes) et **effectuer le pré-traitement des données pour trouver la cellule témoin**.

## Créer de nouveaux résultats

Il faut spécifier le dossier dans lequel on veut sauvegarder les sorties SQAVI et MCMC.

**ATTENTION** : il faut que le dossier ait été créé avec la structure suivante afin de pouvoir enregistrer les tracés :
```
.
├── exemple
    └── plots
```

In [7]:
saveFolder = "results/exemple";

## 7.1. Estimations SQAVI

Configuration SQAVI.

In [12]:
nEpochMax = 10;
epochSize = 1;

spatialScheme = Dict(
    :m => m,
    :Fmu => iGMRF(m₁, m₂, 1, 1),
    :Fphi => iGMRF(m₁, m₂, 1, 1),
    :data => data,
);

Exécution SQAVI

res = runSQAVI(nEpochMax, epochSize, spatialScheme, saveFolder=saveFolder);

Tracés

In [11]:
controlCell = MTLcell; # Numéro de la cellule témoin.

plotConvergenceCriterion(res.MCKL, saveFolder=saveFolder);
plotTraceSQAVI(res.traces[:muMean][controlCell, :], "μₘₜₗ", saveFolder=saveFolder);
plotTraceSQAVI(res.traces[:phiMean][controlCell, :], "ϕₘₜₗ", saveFolder=saveFolder);
plotTraceSQAVI(res.traces[:xiMean], "ξ", saveFolder=saveFolder);
trace = res.traces[:kappaUparams][1, :] ./ res.traces[:kappaUparams][2, :];
plotTraceSQAVI(trace, "κᵤ", saveFolder=saveFolder);
trace = res.traces[:kappaVparams][1, :] ./ res.traces[:kappaVparams][2, :];
plotTraceSQAVI(trace, "κᵥ", saveFolder=saveFolder);

Enregistrement des estimations.

In [20]:
muSQAVI = res.traces[:muMean][:, end];
save_vector("$saveFolder/mu_sqavi_estimates.bin", muSQAVI);
phiSQAVI = res.traces[:phiMean][:, end];
save_vector("$saveFolder/phi_sqavi_estimates.bin", phiSQAVI);

## 7.2. Estimations MCMC

Configuration.

In [17]:
datastructure = Dict(
    :Y => data,
    :Fmu => iGMRF(m₁, m₂, 1, 1),
    :Fphi => iGMRF(m₁, m₂, 1, 1),
);

niter = 1000;

stepsize = Dict(
    :μ => 1,
    :ϕ => .1,
    :ξ => .005,
);

Exécution.

In [ ]:
chain = mcmc(datastructure, niter, initialvalues, stepsize, saveFolder=saveFolder);

Taux d'acceptation.

In [ ]:
changerate(chain)

### Tracés

Modifier la valeur de `varName` pour afficher la trace d'un autre paramètre.

Le tracé apparaîtra dans le dossier de sauvegarde.

In [12]:
varName = "ξ";

plotTraceMCMC(chain, varName, saveFolder=saveFolder);

Enregistrement des estimations.

In [24]:
muMCMC = mean(chain[1001:end, 1:m, :].value[:, :], dims=1)[:];
save_vector("$saveFolder/mu_mcmc_estimates.bin", muMCMC);
phiMCMC = mean(chain[1001:end, m+1:2*m, :].value[:, :], dims=1)[:];
save_vector("$saveFolder/phi_mcmc_estimates.bin", phiMCMC);

## 7.3. Comparaison SQAVI et MCMC

### Tracés de comparaison

On a besoin de spécifier l'itération de chauffe du MCMC (`warmingSize`) pour afficher un échantillon *a posteriori* satisfaisant.

`controlCell` indique la cellule témoin pour laquelle on va regarder $\mu$ et $\phi$, il faut qu'elle soit comprise entre $1$ et $m$.

**ATTENTION** : les domaines de définition des lois d'approximations doivent être spécifiés dans le fichier `src/plotting.jl` pour chaque tracé, sans quoi les densités d'approximation n'apparaîtront pas convenablement.

In [16]:
controlCell = MTLcell;
warmingSize = 1000;

plotSQAVIvsMCMC(controlCell, sqaviRes=res, mcmcChain=chain, warmingSize=warmingSize, saveFolder=saveFolder);

### Estimations MCMC

In [28]:
xi_mcmc = mean(chain[:, "ξ", :].value[warmingSize:end]);
kappa_v_mcmc = mean(chain[:, "κᵥ", :].value[warmingSize:end]);
kappa_u_mcmc = mean(chain[:, "κᵤ", :].value[warmingSize:end]);
mu_mcmc = mean(chain[:, "μ$controlCell", :].value[warmingSize:end]);
phi_mcmc = mean(chain[:, "ϕ$controlCell", :].value[warmingSize:end]);

### Estimations SQAVI

In [29]:
xi_sqavi = res.traces[:xiMean][end];
kappa_v_sqavi = res.traces[:kappaVparams][1, end] ./ res.traces[:kappaVparams][2, end];
kappa_u_sqavi = res.traces[:kappaUparams][1, end] ./ res.traces[:kappaUparams][2, end];
mu_sqavi = res.traces[:muMean][controlCell, end];
phi_sqavi = res.traces[:phiMean][controlCell, end];

### Distance absolue entre les estimations

In [30]:
abs(kappa_v_mcmc - kappa_v_sqavi)

2.2210796322428052

In [19]:
res.approxMarginals[controlCell]

FullNormal(
dim: 2
μ: [44.23616832491034, 2.3821374597410214]
Σ: [2.3694097596689296 0.03447518679825579; 0.03447518679825579 0.00767931765991413]
)


In [ ]:
include("src/plotting.jl")

plotComparisonJointMarginal(
    controlCell,
    sqaviRes=res,
    mcmcChain=chain,
    warmingSize=warmingSize,
    saveFolder=saveFolder,
);

false